<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive-into-Deep-Learning-Notes-/blob/main/10_modernRNNs_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

* RNNs are faced with two numerical stability problems: exploding and vanishing gradients.
* For exploding gradients, the gradient clipping method was introduced in the previous chapter

## 10.1 Long Short-Term Memory (LSTM)

* Long short-term memory (LSTM) was used to address the vanishing gradients in RNNs.
* Recurrent Neural Networks suffer from short-term memory due to vanishing gradient problem, especially when working with large data sequences.
* Newer versions of RNNs can preserve important information from earlier parts of the sequence and carry it forward.

### LSTMs

Standard RNNs and LSTMs use recurrent units to learn from sequential data. But, the structure of the recurrent unit is very different. Recurrent structure in the standard RNNs has the following structure:
$$
𝐇_t = Φ[𝐗_t𝐖_{xh} + 𝐇_{t-1}𝐖_{hh} + 𝐛_h]
$$
LSTM employs various gates to decide what information to keep or discard. Also, it adds a **cell state**, which is like a long term memory of LSTM. The recurrent unit of LSTM is more complex than RNNs and contains the following parts:

* **Hidden state** ($𝐇_{t-1}$) & new inputs ($𝐗_t$): These two are concatenated before going through different gates.

* **Forget gate**: Controls what information should be forgotten.
$$
𝐅_t = σ[𝐗_tW_{xf} + 𝐇_{t-1}𝐖_{hf} + 𝐛_f]
$$
Since sigmoid function ranges between 0 and 1, it sets which value in **the cell state** should be forgotten (multiplied by 0), remembered (multiplied by 1), or partially remembered (multiplied by some value between zero and one).

* **input gate**: Helps to identify important elements that need to be added to cell state.
$$
𝐈_t = σ[𝐗_tW_{xi} + 𝐇_{t-1}𝐖_{hi} + 𝐛_i]
$$
Intuitively, the input gate determines how much of the input node's value should be added to the current memory cell internal state.

* **Input node**:
$$
𝐂̃_t = tanh[𝐗_tW_{xc} + 𝐇_{t-1}𝐖_{hc} + 𝐛_c]
$$

* **Cell state** ($𝐂_t$): First, the previous cell state ($𝐂_{t-1}$) gets multiplied by the result of the forget gate (determines how much of the old cell internal state we retain.) Then we add new information from [input gate * cell state candidate] to get the cell sate (This governs how much we take new data into account.)
$$
𝐂_t = 𝐅_t ⊗ 𝐂_{t-1} + 𝐈_t ⊗ 𝐂̃_t
$$

So, in order to update the hidden state ($𝐇_t$), the follwing formula can be utilized:
$$
𝐇_t = 𝐎_t ⊗ tanh(𝐂_t)
$$

